In [2]:
!pip install -q datasets transformers qwen_vl_utils accelerate


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [1]:
import io
import json
from PIL import Image
from datasets import load_dataset
import torch
from transformers import AutoModelForVision2Seq, AutoProcessor, Qwen2VLProcessor
#from trl import SFTConfig, SFTTrainer
#from peft import LoraConfig
from qwen_vl_utils import process_vision_info



In [2]:
# 시스템(assistant)에게 주어진 역할
system_message = "당신은 이미지로부터 문자와 숫자를 판독하는 전문적인 OCR 모델입니다"

# 실제로 사용자 입력 -> 모델이 답해야 하는 프롬프트
prompt = """입력 정보:
- image: [image]

위 이미지에서 모든 문자,숫자를 판독해서 텍스트로 결과를 출력하세요. 
이미지에 있는 글자 이외에는 어떤응답도 하지마세요.
출력 시 **아래 TEXT 형태**를 반드시 지키세요:
result
원재료명 설탕,물엿,젤라틴,D-소르비톨(감미료),산도조절제I,산도조절제II,산도조절제III,혼합제제I(젓산,젓산칼슘,이산화규소),향료3종,당류가공품,펙틴,산도조절제IV,블랙캐롯농축액,혼합재제II(홍화황색소,치자청색소,주정),천연향료(콜라향)
복숭항,밀,우유,대두,땅콩,달걀,토마토 혼입 가능
복숭항,밀,우유,대두,땅콩,달걀,토마토 와 같은 제조시설에서 제조
"""

In [3]:
import io
from PIL import Image
from IPython.display import display
import base64

def load_and_display_image(file_path):
  """
  로컬 JPG 파일을 불러와서 코랩 출력창에 표시합니다.

  Args:
    file_path (str): 읽어올 JPG 파일의 경로.
  """
  try:
    with open(file_path, "rb") as f:
      image_data = f.read()

    # 이미지 데이터를 메모리 버퍼에 씁니다.
    buffer = io.BytesIO(image_data)

    # 버퍼의 시작점으로 돌아갑니다.
    buffer.seek(0)

    # 버퍼에서 이미지를 엽니다.
    image = Image.open(buffer)

      # 이미지를 Base64 문자열로 인코딩
    buffer_for_b64 = io.BytesIO()
    image.save(buffer_for_b64, format='PNG')
    encoded_string = base64.b64encode(buffer_for_b64.getvalue()).decode('utf-8')

    # IPython.display.display 함수를 사용하여 이미지를 출력합니다.
    #print("이미지를 출력합니다=",len(encoded_string))
    #display(image)

    return image, encoded_string

  except FileNotFoundError:
    print(f"Error: 파일 '{file_path}'을(를) 찾을 수 없습니다.")
  except Exception as e:
    print(f"Error: 이미지를 불러오는 중 오류가 발생했습니다: {e}")



In [ ]:

def format_data(image_path, txt_path):
   #image_path = '/content/김광무_100.jpg'
   #image, encoded_string = load_and_display_image(image_path)

   return {
       "messages": [
           {
               "role": "system",
               "content": [
                   {
                       "type": "text",
                       "text": system_message
                   }
               ],
           },
           {
               "role": "user",
               "content": [
                   {
                       "type": "text",
                       "text": prompt,
                   },                   
                   {
                       "type": "image",
                       "image": image_path,
                   },
               ],
           },
           {
               "role": "assistant",
               "content": [
                   {
                       "type": "text",
                       "text": "",
                   }
               ],
           },
       ],
      "ocr_file_name": txt_path,
   }

In [5]:
import os

folder_path = "./data"
train_list= []

for file_name in os.listdir(folder_path):
  
  if file_name.lower().endswith(".jpg"):
    #print("file_name=",file_name)
    # 파일의 전체 경로를 만듭니다.
    full_path = os.path.join(folder_path, file_name)
    
    # 1. 파일명에서 확장자를 제외한 부분(stem)을 가져옵니다.
    file_stem, _ = os.path.splitext(file_name)
    
    # 2. 새로운 파일명(original_name_ocr.txt)을 만듭니다.
    new_file_name = f"{file_stem}_ocr25.txt"
    
    # 3. 원래 파일의 폴더 경로와 새로운 파일명을 합쳐서 새 경로를 만듭니다.
    txt_path = os.path.join(folder_path, new_file_name)
    
    train_list.append(format_data(full_path, txt_path))
    


In [6]:
print(len(train_list), train_list[2])

62 {'messages': [{'role': 'system', 'content': [{'type': 'text', 'text': '당신은 이미지로부터 문자와 숫자를 판독하는 전문적인 OCR 모델입니다'}]}, {'role': 'user', 'content': [{'type': 'image', 'image': './data/김광무_103.jpg'}]}, {'role': 'assistant', 'content': [{'type': 'text', 'text': ''}]}], 'ocr_file_name': './data/김광무_103_ocr25.txt'}


In [7]:
model_id = "Qwen/Qwen2.5-VL-7B-Instruct"

model = AutoModelForVision2Seq.from_pretrained(
   model_id,
   device_map="auto",                            # GPU 메모리에 자동 할당
   torch_dtype=torch.bfloat16,                   # bfloat16 정밀도 사용
)

processor = AutoProcessor.from_pretrained(model_id)  # 텍스트/이미지 전처리기 로드

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/modeling_auto.py:2242: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

chat_template.json: 0.00B [00:00, ?B/s]

In [8]:
sample_data = train_list[0]['messages']

sample_prompt = processor.apply_chat_template(
    sample_data,
    tokenize=False,
    add_generation_prompt=False,
)

In [9]:
def split_input_and_label(prompt):
    input = prompt.split('<|im_start|>assistant')[0] + '<|im_start|>assistant'
    label = prompt.split('<|im_start|>assistant')[1]
    return input, label

In [10]:
sample_prompt_input, sample_prompt_label = split_input_and_label(sample_prompt)

In [11]:
sample_data_image_inputs, sample_data_video_inputs = process_vision_info(sample_data)

In [12]:
inputs = processor(
    text=[sample_prompt_input],
    images=sample_data_image_inputs,
    videos=sample_data_video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

In [13]:
%%time
# Inference: Generation of the output
generated_ids = model.generate(
    **inputs,
    max_new_tokens=2048,
    temperature=0.1,
    top_p=0.001,
    repetition_penalty=1.05,
)

generated_ids_trimmed = [out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)]
output_text = processor.batch_decode(generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False)
print("추론결과=\n")
print(output_text)


추론결과=

['\n이미지에는 SPAM 브랜드의 제품이 담겨 있습니다. 제품의 내용물은 돼지고기이며, 그 외에도 다양한 성분이 포함되어 있습니다. 제품의 용량은 605g이고, 영양 정보는 다음과 같습니다:\n\n- 총 내용량: 605g / 100g 당 285kcal\n- 나트륨: 930mg (47%)\n- 탄수화물: 8g (2%)\n- 당류: 1g (미만 1%)\n- 지방: 23g (43%)\n- 트랜스지방: 0g\n- 포화지방: 7g (47%)\n- 콜레스테롤: 50mg (17%)\n- 단백질: 12g (22%)\n\n또한, 제품의 유통기한은 25.09.2023까지입니다.']
CPU times: user 12.2 s, sys: 581 ms, total: 12.8 s
Wall time: 10.3 s


In [14]:
print(output_text[0])


이미지에는 SPAM 브랜드의 제품이 담겨 있습니다. 제품의 내용물은 돼지고기이며, 그 외에도 다양한 성분이 포함되어 있습니다. 제품의 용량은 605g이고, 영양 정보는 다음과 같습니다:

- 총 내용량: 605g / 100g 당 285kcal
- 나트륨: 930mg (47%)
- 탄수화물: 8g (2%)
- 당류: 1g (미만 1%)
- 지방: 23g (43%)
- 트랜스지방: 0g
- 포화지방: 7g (47%)
- 콜레스테롤: 50mg (17%)
- 단백질: 12g (22%)

또한, 제품의 유통기한은 25.09.2023까지입니다.


In [15]:
%%time
from tqdm import tqdm

for index, item in enumerate(tqdm(train_list)):
    test_data = item['messages']
    txt_path = item['ocr_file_name']
    #print("처리중 txt_path=", txt_path)

    test_prompt = processor.apply_chat_template(
        test_data,
        tokenize=False,
        add_generation_prompt=False,
    )
    
    test_prompt_input, test_prompt_label = split_input_and_label(test_prompt)
    test_data_image_inputs, test_data_video_inputs = process_vision_info(test_data)
    
    inputs = processor(
        text=[test_prompt_input],
        images=test_data_image_inputs,
        videos=test_data_video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to("cuda")
    
    # Inference: Generation of the output
    generated_ids = model.generate(
        **inputs,
        max_new_tokens=2048,
        temperature=0.1,
        top_p=0.001,
        repetition_penalty=1.05,
    )

    generated_ids_trimmed = [out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)]
    output_text = processor.batch_decode(generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False)
    #print("추론결과=\n")
    #print(output_text)    
    
    # 새 .txt 파일을 생성하고 내용을 씁니다.
    with open(txt_path, "w", encoding="utf-8") as f:
        f.write(output_text[0])
        f.close()

    #print(f"'{txt_path}' 파일이 생성되었습니다.")   

100%|██████████| 62/62 [19:21<00:00, 18.73s/it]

CPU times: user 26min 29s, sys: 4min 22s, total: 30min 52s
Wall time: 19min 21s
